In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('nhl_stats.csv', index_col=0)
df=df.dropna()
df['opponent'] = df.date.apply(lambda x: x.split()[1])

abb_to_team = {'PIT':'Pittsburgh Penguins', 'PHI':'Philadelphia Flyers',
'TOR':'Toronto Maple Leafs', 'MTL':'Montréal Canadiens', 'MIN':'Minnesota Wild',
'COL':'Colorado Avalanche', 'EDM':'Edmonton Oilers', 'CHI':'Chicago Blackhawks',
'CGY':'Calgary Flames', 'CAR':'Carolina Hurricanes', 'DAL':'Dallas Stars',
'OTT':'Ottawa Senators', 'BUF':'Buffalo Sabres', 'WSH':'Washington Capitals',
'ATL':'Atlanta Thrashers', 'ANA':'Anaheim Ducks', 'DET':'Detroit Red Wings',
'NJD':'New Jersey Devils','CBJ':'Columbus Blue Jackets', 'SJS':'San Jose Sharks',
'STL':'St. Louis Blues', 'PHX':'Phoenix Coyotes', 'NYR':'New York Rangers',
'TBL':'Tampa Bay Lightning', 'LAK':'Los Angeles Kings', 'NSH':'Nashville Predators',
'NYI':'New York Islanders', 'VAN':'Vancouver Canucks', 'BOS':'Boston Bruins',
'FLA':'Florida Panthers', 'WPG':'Winnipeg Jets', 'ARI':'Arizona Coyotes',
'VGK':'Vegas Golden Knights', 'SEA':'Seattle Kraken'}

df['identifier'] = df['date'].apply(lambda x: x.split()[0][10:])
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x[0:10], format='%Y/%m/%d'))
df['opponent'] = df['opponent'].apply(lambda x: abb_to_team[x])
df.head()

,team,date,points,RW,ROW,SOW,goals,goals_against,power_play,penalty_kill,net_ppp,net_pkp,shots,shots_against,FOWp,opponent,identifier
0,New Jersey Devils,2005-10-05,2,1,1,0,5,1,25.0,90.9,25.0,90.9,37.0,37.0,57.1,Pittsburgh Penguins,vs
1,New York Islanders,2005-10-05,0,0,0,0,4,6,14.3,81.8,14.3,81.8,26.0,39.0,51.5,Buffalo Sabres,@
2,New York Rangers,2005-10-05,2,1,1,0,5,3,50.0,90.0,50.0,90.0,27.0,27.0,51.6,Philadelphia Flyers,@
3,Philadelphia Flyers,2005-10-05,0,0,0,0,3,5,10.0,50.0,10.0,50.0,27.0,27.0,48.4,New York Rangers,vs
4,Pittsburgh Penguins,2005-10-05,0,0,0,0,1,5,9.1,75.0,9.1,75.0,37.0,37.0,42.9,New Jersey Devils,@


In [3]:
df = df.astype({'goals':int, 'goals_against':int, 'shots':float, 'shots_against':float, 'FOWp':float})
df.drop(columns=['RW', 'ROW', 'SOW', 'net_ppp', 'net_pkp'], inplace=True)
df.head()

,team,date,points,goals,goals_against,power_play,penalty_kill,shots,shots_against,FOWp,opponent,identifier
0,New Jersey Devils,2005-10-05,2,5,1,25.0,90.9,37.0,37.0,57.1,Pittsburgh Penguins,vs
1,New York Islanders,2005-10-05,0,4,6,14.3,81.8,26.0,39.0,51.5,Buffalo Sabres,@
2,New York Rangers,2005-10-05,2,5,3,50.0,90.0,27.0,27.0,51.6,Philadelphia Flyers,@
3,Philadelphia Flyers,2005-10-05,0,3,5,10.0,50.0,27.0,27.0,48.4,New York Rangers,vs
4,Pittsburgh Penguins,2005-10-05,0,1,5,9.1,75.0,37.0,37.0,42.9,New Jersey Devils,@


In [4]:
try:
    df['penalty_kill'] = pd.to_numeric(df['penalty_kill'])
except ValueError:
    mask = pd.to_numeric(df['penalty_kill'], errors='coerce').isna()
    L = df.loc[mask, 'penalty_kill'].tolist()
    print("Not converted values are: " + ", ".join(L))

Not converted values are: --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --, --

In [5]:
df['penalty_kill'].replace('--', 80.0, inplace=True)
df['power_play'].replace('--', 20.0, inplace=True)
df['penalty_kill'] = pd.to_numeric(df.penalty_kill)
df['power_play'] = pd.to_numeric(df.power_play)
df.dtypes

team                     object
date             datetime64[ns]
points                    int64
goals                     int32
goals_against             int32
power_play              float64
penalty_kill            float64
shots                   float64
shots_against           float64
FOWp                    float64
opponent                 object
identifier               object
dtype: object

In [6]:
df_home = df[df['identifier'] == 'vs']
df_home.drop(columns=['identifier'], inplace=True)
df_away = df[df['identifier'] == '@']
df_away.drop(columns=['identifier'], inplace=True)

c:\Users\cbarg\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
merged_df = pd.merge(df_home, df_away, how='left', left_on=['date', 'opponent'], right_on=['date', 'team'])
merged_df.drop(columns=['opponent_x', 'opponent_y'], inplace=True)
merged_df.head()

,team_x,date,points_x,goals_x,goals_against_x,power_play_x,penalty_kill_x,shots_x,shots_against_x,FOWp_x,team_y,points_y,goals_y,goals_against_y,power_play_y,penalty_kill_y,shots_y,shots_against_y,FOWp_y
0,New Jersey Devils,2005-10-05,2,5,1,25.0,90.9,37.0,37.0,57.1,Pittsburgh Penguins,0,1,5,9.1,75.0,37.0,37.0,42.9
1,Philadelphia Flyers,2005-10-05,0,3,5,10.0,50.0,27.0,27.0,48.4,New York Rangers,2,5,3,50.0,90.0,27.0,27.0,51.6
2,Boston Bruins,2005-10-05,0,1,2,0.0,80.0,30.0,21.0,56.5,Montréal Canadiens,2,2,1,20.0,100.0,21.0,30.0,43.6
3,Buffalo Sabres,2005-10-05,2,6,4,18.2,85.7,39.0,26.0,48.5,New York Islanders,0,4,6,14.3,81.8,26.0,39.0,51.5
4,Toronto Maple Leafs,2005-10-05,1,2,2,9.1,100.0,25.0,23.0,47.5,Ottawa Senators,2,2,2,0.0,90.9,23.0,25.0,52.5


How to calculate rolling stats with a window of 5 shown here.

In [12]:
SHORT_WINDOW = 20
MEDIUM_WINDOW = 40
LONG_WINDOW = 60

df_devils = df[df.team == 'New Jersey Devils']
df_devils['rolling_points_5'] = df_devils['points'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_points_10'] = df_devils['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_devils['rolling_points_20'] = df_devils['points'].rolling(LONG_WINDOW).mean().shift()
df_devils['rolling_goals_5'] = df_devils['goals'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_goals_10'] = df_devils['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_devils['rolling_ga_5'] = df_devils['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_ga_10'] = df_devils['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_devils['rolling_pp_5'] = df_devils['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_pk_5'] = df_devils['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_shots_5'] = df_devils['shots'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_sa_5'] = df_devils['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_fow_5'] = df_devils['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_devils['rolling_ga_sum'] = df_devils['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_devils['rolling_sa_sum'] = df_devils['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_devils['rolling_svp_5'] = (df_devils.rolling_sa_sum - df_devils.rolling_ga_sum) / df_devils.rolling_sa_sum
df_devils.to_csv('.//Team Frames//devils.csv', index=False)

df_penguins = df[df.team == 'Pittsburgh Penguins']
df_penguins['rolling_points_5'] = df_penguins['points'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_points_10'] = df_penguins['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_penguins['rolling_points_20'] = df_penguins['points'].rolling(LONG_WINDOW).mean().shift()
df_penguins['rolling_goals_5'] = df_penguins['goals'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_goals_10'] = df_penguins['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_penguins['rolling_ga_5'] = df_penguins['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_ga_10'] = df_penguins['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_penguins['rolling_pp_5'] = df_penguins['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_pk_5'] = df_devils['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_shots_5'] = df_penguins['shots'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_sa_5'] = df_penguins['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_fow_5'] = df_penguins['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_penguins['rolling_ga_sum'] = df_penguins['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_penguins['rolling_sa_sum'] = df_penguins['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_penguins['rolling_svp_5'] = (df_penguins.rolling_sa_sum - df_penguins.rolling_ga_sum) / df_penguins.rolling_sa_sum
df_penguins.to_csv('.//Team Frames//penguins.csv', index=False)

df_flyers = df[df.team == 'Philadelphia Flyers']
df_flyers['rolling_points_5'] = df_flyers['points'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_points_10'] = df_flyers['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_flyers['rolling_points_20'] = df_flyers['points'].rolling(LONG_WINDOW).mean().shift()
df_flyers['rolling_goals_5'] = df_flyers['goals'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_goals_10'] = df_flyers['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_flyers['rolling_ga_5'] = df_flyers['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_ga_10'] = df_flyers['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_flyers['rolling_pp_5'] = df_flyers['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_pk_5'] = df_flyers['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_shots_5'] = df_flyers['shots'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_sa_5'] = df_flyers['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_fow_5'] = df_flyers['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_flyers['rolling_ga_sum'] = df_flyers['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_flyers['rolling_sa_sum'] = df_flyers['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_flyers['rolling_svp_5'] = (df_flyers.rolling_sa_sum - df_flyers.rolling_ga_sum) / df_flyers.rolling_sa_sum
df_flyers.to_csv('.//Team Frames//flyers.csv', index=False)

df_leafs = df[df.team == 'Toronto Maple Leafs']
df_leafs['rolling_points_5'] = df_leafs['points'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_points_10'] = df_leafs['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_leafs['rolling_points_20'] = df_leafs['points'].rolling(LONG_WINDOW).mean().shift()
df_leafs['rolling_goals_5'] = df_leafs['goals'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_goals_10'] = df_leafs['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_leafs['rolling_ga_5'] = df_leafs['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_ga_10'] = df_leafs['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_leafs['rolling_pp_5'] = df_leafs['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_pk_5'] = df_leafs['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_shots_5'] = df_leafs['shots'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_sa_5'] = df_leafs['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_fow_5'] = df_leafs['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_leafs['rolling_ga_sum'] = df_leafs['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_leafs['rolling_sa_sum'] = df_leafs['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_leafs['rolling_svp_5'] = (df_leafs.rolling_sa_sum - df_leafs.rolling_ga_sum) / df_leafs.rolling_sa_sum
df_leafs.to_csv('.//Team Frames//leafs.csv', index=False)

df_canadiens = df[df.team == 'Montréal Canadiens']
df_canadiens['rolling_points_5'] = df_canadiens['points'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_points_10'] = df_canadiens['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_canadiens['rolling_points_20'] = df_canadiens['points'].rolling(LONG_WINDOW).mean().shift()
df_canadiens['rolling_goals_5'] = df_canadiens['goals'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_goals_10'] = df_canadiens['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_canadiens['rolling_ga_5'] = df_canadiens['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_ga_10'] = df_canadiens['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_canadiens['rolling_pp_5'] = df_canadiens['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_pk_5'] = df_canadiens['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_shots_5'] = df_canadiens['shots'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_sa_5'] = df_canadiens['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_fow_5'] = df_canadiens['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_canadiens['rolling_ga_sum'] = df_canadiens['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_canadiens['rolling_sa_sum'] = df_canadiens['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_canadiens['rolling_svp_5'] = (df_canadiens.rolling_sa_sum - df_canadiens.rolling_ga_sum) / df_canadiens.rolling_sa_sum
df_canadiens.to_csv('.//Team Frames//canadiens.csv', index=False)

df_wild = df[df.team == 'Minnesota Wild']
df_wild['rolling_points_5'] = df_wild['points'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_points_10'] = df_wild['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_wild['rolling_points_20'] = df_wild['points'].rolling(LONG_WINDOW).mean().shift()
df_wild['rolling_goals_5'] = df_wild['goals'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_goals_10'] = df_wild['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_wild['rolling_ga_5'] = df_wild['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_ga_10'] = df_wild['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_wild['rolling_pp_5'] = df_wild['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_pk_5'] = df_wild['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_shots_5'] = df_wild['shots'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_sa_5'] = df_wild['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_fow_5'] = df_wild['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_wild['rolling_ga_sum'] = df_wild['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_wild['rolling_sa_sum'] = df_wild['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_wild['rolling_svp_5'] = (df_wild.rolling_sa_sum - df_wild.rolling_ga_sum) / df_wild.rolling_sa_sum
df_wild.to_csv('.//Team Frames//wild.csv', index=False)

df_avalanche = df[df.team == 'Colorado Avalanche']
df_avalanche['rolling_points_5'] = df_avalanche['points'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_points_10'] = df_avalanche['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_avalanche['rolling_points_20'] = df_avalanche['points'].rolling(LONG_WINDOW).mean().shift()
df_avalanche['rolling_goals_5'] = df_avalanche['goals'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_goals_10'] = df_avalanche['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_avalanche['rolling_ga_5'] = df_avalanche['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_ga_10'] = df_avalanche['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_avalanche['rolling_pp_5'] = df_avalanche['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_pk_5'] = df_avalanche['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_shots_5'] = df_avalanche['shots'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_sa_5'] = df_avalanche['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_fow_5'] = df_avalanche['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_avalanche['rolling_ga_sum'] = df_avalanche['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_avalanche['rolling_sa_sum'] = df_avalanche['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_avalanche['rolling_svp_5'] = (df_avalanche.rolling_sa_sum - df_avalanche.rolling_ga_sum) / df_avalanche.rolling_sa_sum
df_avalanche.to_csv('.//Team Frames//avalanche.csv', index=False)

df_oilers = df[df.team == 'Edmonton Oilers']
df_oilers['rolling_points_5'] = df_oilers['points'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_points_10'] = df_oilers['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_oilers['rolling_points_20'] = df_oilers['points'].rolling(LONG_WINDOW).mean().shift()
df_oilers['rolling_goals_5'] = df_oilers['goals'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_goals_10'] = df_oilers['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_oilers['rolling_ga_5'] = df_oilers['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_ga_10'] = df_oilers['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_oilers['rolling_pp_5'] = df_oilers['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_pk_5'] = df_oilers['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_shots_5'] = df_oilers['shots'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_sa_5'] = df_oilers['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_fow_5'] = df_oilers['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_oilers['rolling_ga_sum'] = df_oilers['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_oilers['rolling_sa_sum'] = df_oilers['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_oilers['rolling_svp_5'] = (df_oilers.rolling_sa_sum - df_oilers.rolling_ga_sum) / df_oilers.rolling_sa_sum
df_oilers.to_csv('.//Team Frames//oilers.csv', index=False)

df_blackhawks = df[df.team == 'Chicago Blackhawks']
df_blackhawks['rolling_points_5'] = df_blackhawks['points'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_points_10'] = df_blackhawks['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_blackhawks['rolling_points_20'] = df_blackhawks['points'].rolling(LONG_WINDOW).mean().shift()
df_blackhawks['rolling_goals_5'] = df_blackhawks['goals'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_goals_10'] = df_blackhawks['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_blackhawks['rolling_ga_5'] = df_blackhawks['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_ga_10'] = df_blackhawks['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_blackhawks['rolling_pp_5'] = df_blackhawks['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_pk_5'] = df_blackhawks['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_shots_5'] = df_blackhawks['shots'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_sa_5'] = df_blackhawks['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_fow_5'] = df_blackhawks['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_blackhawks['rolling_ga_sum'] = df_blackhawks['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_blackhawks['rolling_sa_sum'] = df_blackhawks['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_blackhawks['rolling_svp_5'] = (df_blackhawks.rolling_sa_sum - df_blackhawks.rolling_ga_sum) / df_blackhawks.rolling_sa_sum
df_blackhawks.to_csv('.//Team Frames//blackhawks.csv', index=False)

df_flames = df[df.team == 'Calgary Flames']
df_flames['rolling_points_5'] = df_flames['points'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_points_10'] = df_flames['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_flames['rolling_points_20'] = df_flames['points'].rolling(LONG_WINDOW).mean().shift()
df_flames['rolling_goals_5'] = df_flames['goals'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_goals_10'] = df_flames['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_flames['rolling_ga_5'] = df_flames['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_ga_10'] = df_flames['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_flames['rolling_pp_5'] = df_flames['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_pk_5'] = df_flames['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_shots_5'] = df_flames['shots'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_sa_5'] = df_flames['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_fow_5'] = df_flames['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_flames['rolling_ga_sum'] = df_flames['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_flames['rolling_sa_sum'] = df_flames['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_flames['rolling_svp_5'] = (df_flames.rolling_sa_sum - df_flames.rolling_ga_sum) / df_flames.rolling_sa_sum
df_flames.to_csv('.//Team Frames//flames.csv', index=False)

df_hurricanes = df[df.team == 'Carolina Hurricanes']
df_hurricanes['rolling_points_5'] = df_hurricanes['points'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_points_10'] = df_hurricanes['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_hurricanes['rolling_points_20'] = df_hurricanes['points'].rolling(LONG_WINDOW).mean().shift()
df_hurricanes['rolling_goals_5'] = df_hurricanes['goals'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_goals_10'] = df_hurricanes['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_hurricanes['rolling_ga_5'] = df_hurricanes['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_ga_10'] = df_hurricanes['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_hurricanes['rolling_pp_5'] = df_hurricanes['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_pk_5'] = df_hurricanes['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_shots_5'] = df_hurricanes['shots'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_sa_5'] = df_hurricanes['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_fow_5'] = df_hurricanes['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_hurricanes['rolling_ga_sum'] = df_hurricanes['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_hurricanes['rolling_sa_sum'] = df_hurricanes['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_hurricanes['rolling_svp_5'] = (df_hurricanes.rolling_sa_sum - df_hurricanes.rolling_ga_sum) / df_hurricanes.rolling_sa_sum
df_hurricanes.to_csv('.//Team Frames//hurricanes.csv', index=False)

df_stars = df[df.team == 'Dallas Stars']
df_stars['rolling_points_5'] = df_stars['points'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_points_10'] = df_stars['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_stars['rolling_points_20'] = df_stars['points'].rolling(LONG_WINDOW).mean().shift()
df_stars['rolling_goals_5'] = df_stars['goals'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_goals_10'] = df_stars['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_stars['rolling_ga_5'] = df_stars['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_ga_10'] = df_stars['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_stars['rolling_pp_5'] = df_stars['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_pk_5'] = df_stars['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_shots_5'] = df_stars['shots'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_sa_5'] = df_stars['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_fow_5'] = df_stars['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_stars['rolling_ga_sum'] = df_stars['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_stars['rolling_sa_sum'] = df_stars['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_stars['rolling_svp_5'] = (df_stars.rolling_sa_sum - df_stars.rolling_ga_sum) / df_stars.rolling_sa_sum
df_stars.to_csv('.//Team Frames//stars.csv', index=False)

df_senators = df[df.team == 'Ottawa Senators']
df_senators['rolling_points_5'] = df_senators['points'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_points_10'] = df_senators['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_senators['rolling_points_20'] = df_senators['points'].rolling(LONG_WINDOW).mean().shift()
df_senators['rolling_goals_5'] = df_senators['goals'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_goals_10'] = df_senators['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_senators['rolling_ga_5'] = df_senators['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_ga_10'] = df_senators['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_senators['rolling_pp_5'] = df_senators['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_pk_5'] = df_senators['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_shots_5'] = df_senators['shots'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_sa_5'] = df_senators['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_fow_5'] = df_senators['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_senators['rolling_ga_sum'] = df_senators['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_senators['rolling_sa_sum'] = df_senators['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_senators['rolling_svp_5'] = (df_senators.rolling_sa_sum - df_senators.rolling_ga_sum) / df_senators.rolling_sa_sum
df_senators.to_csv('.//Team Frames//senators.csv', index=False)

df_sabres = df[df.team == 'Buffalo Sabres']
df_sabres['rolling_points_5'] = df_sabres['points'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_points_10'] = df_sabres['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_sabres['rolling_points_20'] = df_sabres['points'].rolling(LONG_WINDOW).mean().shift()
df_sabres['rolling_goals_5'] = df_sabres['goals'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_goals_10'] = df_sabres['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_sabres['rolling_ga_5'] = df_sabres['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_ga_10'] = df_sabres['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_sabres['rolling_pp_5'] = df_sabres['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_pk_5'] = df_sabres['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_shots_5'] = df_sabres['shots'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_sa_5'] = df_sabres['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_fow_5'] = df_sabres['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_sabres['rolling_ga_sum'] = df_sabres['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_sabres['rolling_sa_sum'] = df_sabres['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_sabres['rolling_svp_5'] = (df_sabres.rolling_sa_sum - df_sabres.rolling_ga_sum) / df_sabres.rolling_sa_sum
df_sabres.to_csv('.//Team Frames//sabres.csv', index=False)

df_capitals = df[df.team == 'Washington Capitals']
df_capitals['rolling_points_5'] = df_capitals['points'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_points_10'] = df_capitals['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_capitals['rolling_points_20'] = df_capitals['points'].rolling(LONG_WINDOW).mean().shift()
df_capitals['rolling_goals_5'] = df_capitals['goals'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_goals_10'] = df_capitals['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_capitals['rolling_ga_5'] = df_capitals['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_ga_10'] = df_capitals['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_capitals['rolling_pp_5'] = df_capitals['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_pk_5'] = df_capitals['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_shots_5'] = df_capitals['shots'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_sa_5'] = df_capitals['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_fow_5'] = df_capitals['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_capitals['rolling_ga_sum'] = df_capitals['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_capitals['rolling_sa_sum'] = df_capitals['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_capitals['rolling_svp_5'] = (df_capitals.rolling_sa_sum - df_capitals.rolling_ga_sum) / df_capitals.rolling_sa_sum
df_capitals.to_csv('.//Team Frames//capitals.csv', index=False)

df_thrashers = df[df.team == 'Atlanta Thrashers']
df_thrashers['rolling_points_5'] = df_thrashers['points'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_points_10'] = df_thrashers['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_thrashers['rolling_points_20'] = df_thrashers['points'].rolling(LONG_WINDOW).mean().shift()
df_thrashers['rolling_goals_5'] = df_thrashers['goals'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_goals_10'] = df_thrashers['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_thrashers['rolling_ga_5'] = df_thrashers['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_ga_10'] = df_thrashers['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_thrashers['rolling_pp_5'] = df_thrashers['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_pk_5'] = df_thrashers['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_shots_5'] = df_thrashers['shots'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_sa_5'] = df_thrashers['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_fow_5'] = df_thrashers['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_thrashers['rolling_ga_sum'] = df_thrashers['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_thrashers['rolling_sa_sum'] = df_thrashers['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_thrashers['rolling_svp_5'] = (df_thrashers.rolling_sa_sum - df_thrashers.rolling_ga_sum) / df_thrashers.rolling_sa_sum
df_thrashers.to_csv('.//Team Frames//thrashers.csv', index=False)

df_ducks = df[df.team == 'Anaheim Ducks']
df_ducks['rolling_points_5'] = df_ducks['points'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_points_10'] = df_ducks['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_ducks['rolling_points_20'] = df_ducks['points'].rolling(LONG_WINDOW).mean().shift()
df_ducks['rolling_goals_5'] = df_ducks['goals'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_goals_10'] = df_ducks['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_ducks['rolling_ga_5'] = df_ducks['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_ga_10'] = df_ducks['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_ducks['rolling_pp_5'] = df_ducks['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_pk_5'] = df_ducks['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_shots_5'] = df_ducks['shots'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_sa_5'] = df_ducks['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_fow_5'] = df_ducks['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_ducks['rolling_ga_sum'] = df_ducks['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_ducks['rolling_sa_sum'] = df_ducks['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_ducks['rolling_svp_5'] = (df_ducks.rolling_sa_sum - df_ducks.rolling_ga_sum) / df_ducks.rolling_sa_sum
df_ducks.to_csv('.//Team Frames//ducks.csv', index=False)

df_redwings = df[df.team == 'Detroit Red Wings']
df_redwings['rolling_points_5'] = df_redwings['points'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_points_10'] = df_redwings['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_redwings['rolling_points_20'] = df_redwings['points'].rolling(LONG_WINDOW).mean().shift()
df_redwings['rolling_goals_5'] = df_redwings['goals'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_goals_10'] = df_redwings['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_redwings['rolling_ga_5'] = df_redwings['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_ga_10'] = df_redwings['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_redwings['rolling_pp_5'] = df_redwings['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_pk_5'] = df_redwings['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_shots_5'] = df_redwings['shots'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_sa_5'] = df_redwings['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_fow_5'] = df_redwings['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_redwings['rolling_ga_sum'] = df_redwings['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_redwings['rolling_sa_sum'] = df_redwings['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_redwings['rolling_svp_5'] = (df_redwings.rolling_sa_sum - df_redwings.rolling_ga_sum) / df_redwings.rolling_sa_sum
df_redwings.to_csv('.//Team Frames//redwings.csv', index=False)

df_bluejackets = df[df.team == 'Columbus Blue Jackets']
df_bluejackets['rolling_points_5'] = df_bluejackets['points'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_points_10'] = df_bluejackets['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_bluejackets['rolling_points_20'] = df_bluejackets['points'].rolling(LONG_WINDOW).mean().shift()
df_bluejackets['rolling_goals_5'] = df_bluejackets['goals'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_goals_10'] = df_bluejackets['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_bluejackets['rolling_ga_5'] = df_bluejackets['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_ga_10'] = df_bluejackets['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_bluejackets['rolling_pp_5'] = df_bluejackets['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_pk_5'] = df_bluejackets['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_shots_5'] = df_bluejackets['shots'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_sa_5'] = df_bluejackets['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_fow_5'] = df_senators['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_bluejackets['rolling_ga_sum'] = df_bluejackets['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_bluejackets['rolling_sa_sum'] = df_bluejackets['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_bluejackets['rolling_svp_5'] = (df_bluejackets.rolling_sa_sum - df_bluejackets.rolling_ga_sum) / df_bluejackets.rolling_sa_sum
df_bluejackets.to_csv('.//Team Frames//bluejackets.csv', index=False)

df_sharks = df[df.team == 'San Jose Sharks']
df_sharks['rolling_points_5'] = df_sharks['points'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_points_10'] = df_sharks['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_sharks['rolling_points_20'] = df_sharks['points'].rolling(LONG_WINDOW).mean().shift()
df_sharks['rolling_goals_5'] = df_sharks['goals'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_goals_10'] = df_sharks['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_sharks['rolling_ga_5'] = df_sharks['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_ga_10'] = df_sharks['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_sharks['rolling_pp_5'] = df_sharks['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_pk_5'] = df_sharks['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_shots_5'] = df_sharks['shots'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_sa_5'] = df_sharks['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_fow_5'] = df_sharks['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_sharks['rolling_ga_sum'] = df_sharks['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_sharks['rolling_sa_sum'] = df_sharks['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_sharks['rolling_svp_5'] = (df_sharks.rolling_sa_sum - df_sharks.rolling_ga_sum) / df_sharks.rolling_sa_sum
df_sharks.to_csv('.//Team Frames//sharks.csv', index=False)

df_blues = df[df.team == 'St. Louis Blues']
df_blues['rolling_points_5'] = df_blues['points'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_points_10'] = df_blues['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_blues['rolling_points_20'] = df_blues['points'].rolling(LONG_WINDOW).mean().shift()
df_blues['rolling_goals_5'] = df_blues['goals'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_goals_10'] = df_blues['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_blues['rolling_ga_5'] = df_blues['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_ga_10'] = df_blues['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_blues['rolling_pp_5'] = df_blues['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_pk_5'] = df_blues['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_shots_5'] = df_blues['shots'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_sa_5'] = df_blues['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_fow_5'] = df_blues['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_blues['rolling_ga_sum'] = df_blues['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_blues['rolling_sa_sum'] = df_blues['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_blues['rolling_svp_5'] = (df_blues.rolling_sa_sum - df_blues.rolling_ga_sum) / df_blues.rolling_sa_sum
df_blues.to_csv('.//Team Frames//blues.csv', index=False)

df_coyotes = df[(df.team == 'Phoenix Coyotes') | (df.team == 'Arizona Coyotes')]
df_coyotes['rolling_points_5'] = df_coyotes['points'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_points_10'] = df_coyotes['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_coyotes['rolling_points_20'] = df_coyotes['points'].rolling(LONG_WINDOW).mean().shift()
df_coyotes['rolling_goals_5'] = df_coyotes['goals'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_goals_10'] = df_coyotes['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_coyotes['rolling_ga_5'] = df_coyotes['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_ga_10'] = df_coyotes['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_coyotes['rolling_pp_5'] = df_coyotes['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_pk_5'] = df_coyotes['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_shots_5'] = df_coyotes['shots'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_sa_5'] = df_coyotes['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_fow_5'] = df_coyotes['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_coyotes['rolling_ga_sum'] = df_coyotes['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_coyotes['rolling_sa_sum'] = df_coyotes['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_coyotes['rolling_svp_5'] = (df_coyotes.rolling_sa_sum - df_coyotes.rolling_ga_sum) / df_coyotes.rolling_sa_sum
df_coyotes.to_csv('.//Team Frames//coyotes.csv', index=False)

df_rangers = df[df.team == 'New York Rangers']
df_rangers['rolling_points_5'] = df_rangers['points'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_points_10'] = df_rangers['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_rangers['rolling_points_20'] = df_rangers['points'].rolling(LONG_WINDOW).mean().shift()
df_rangers['rolling_goals_5'] = df_rangers['goals'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_goals_10'] = df_rangers['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_rangers['rolling_ga_5'] = df_rangers['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_ga_10'] = df_rangers['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_rangers['rolling_pp_5'] = df_rangers['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_pk_5'] = df_rangers['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_shots_5'] = df_rangers['shots'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_sa_5'] = df_rangers['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_fow_5'] = df_rangers['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_rangers['rolling_ga_sum'] = df_rangers['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_rangers['rolling_sa_sum'] = df_rangers['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_rangers['rolling_svp_5'] = (df_rangers.rolling_sa_sum - df_rangers.rolling_ga_sum) / df_rangers.rolling_sa_sum
df_rangers.to_csv('.//Team Frames//rangers.csv', index=False)

df_lightning = df[df.team == 'Tampa Bay Lightning']
df_lightning['rolling_points_5'] = df_lightning['points'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_points_10'] = df_lightning['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_lightning['rolling_points_20'] = df_lightning['points'].rolling(LONG_WINDOW).mean().shift()
df_lightning['rolling_goals_5'] = df_lightning['goals'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_goals_10'] = df_lightning['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_lightning['rolling_ga_5'] = df_lightning['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_ga_10'] = df_lightning['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_lightning['rolling_pp_5'] = df_lightning['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_pk_5'] = df_lightning['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_shots_5'] = df_lightning['shots'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_sa_5'] = df_lightning['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_fow_5'] = df_lightning['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_lightning['rolling_ga_sum'] = df_lightning['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_lightning['rolling_sa_sum'] = df_lightning['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_lightning['rolling_svp_5'] = (df_lightning.rolling_sa_sum - df_lightning.rolling_ga_sum) / df_lightning.rolling_sa_sum
df_lightning.to_csv('.//Team Frames//lightning.csv', index=False)

df_kings = df[df.team == 'Los Angeles Kings']
df_kings['rolling_points_5'] = df_kings['points'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_points_10'] = df_kings['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_kings['rolling_points_20'] = df_kings['points'].rolling(LONG_WINDOW).mean().shift()
df_kings['rolling_goals_5'] = df_kings['goals'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_goals_10'] = df_kings['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_kings['rolling_ga_5'] = df_kings['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_ga_10'] = df_kings['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_kings['rolling_pp_5'] = df_kings['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_pk_5'] = df_kings['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_shots_5'] = df_kings['shots'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_sa_5'] = df_kings['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_fow_5'] = df_kings['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_kings['rolling_ga_sum'] = df_kings['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_kings['rolling_sa_sum'] = df_kings['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_kings['rolling_svp_5'] = (df_kings.rolling_sa_sum - df_kings.rolling_ga_sum) / df_kings.rolling_sa_sum
df_kings.to_csv('.//Team Frames//kings.csv', index=False)

df_predators = df[df.team == 'Nashville Predators']
df_predators['rolling_points_5'] = df_predators['points'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_points_10'] = df_predators['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_predators['rolling_points_20'] = df_predators['points'].rolling(LONG_WINDOW).mean().shift()
df_predators['rolling_goals_5'] = df_predators['goals'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_goals_10'] = df_predators['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_predators['rolling_ga_5'] = df_predators['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_ga_10'] = df_predators['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_predators['rolling_pp_5'] = df_predators['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_pk_5'] = df_predators['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_shots_5'] = df_predators['shots'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_sa_5'] = df_predators['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_fow_5'] = df_predators['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_predators['rolling_ga_sum'] = df_predators['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_predators['rolling_sa_sum'] = df_predators['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_predators['rolling_svp_5'] = (df_predators.rolling_sa_sum - df_predators.rolling_ga_sum) / df_predators.rolling_sa_sum
df_predators.to_csv('.//Team Frames//predators.csv', index=False)

df_islanders = df[df.team == 'New York Islanders']
df_islanders['rolling_points_5'] = df_islanders['points'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_points_10'] = df_islanders['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_islanders['rolling_points_20'] = df_islanders['points'].rolling(LONG_WINDOW).mean().shift()
df_islanders['rolling_goals_5'] = df_islanders['goals'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_goals_10'] = df_islanders['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_islanders['rolling_ga_5'] = df_islanders['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_ga_10'] = df_islanders['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_islanders['rolling_pp_5'] = df_islanders['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_pk_5'] = df_islanders['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_shots_5'] = df_islanders['shots'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_sa_5'] = df_islanders['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_fow_5'] = df_islanders['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_islanders['rolling_ga_sum'] = df_islanders['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_islanders['rolling_sa_sum'] = df_islanders['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_islanders['rolling_svp_5'] = (df_islanders.rolling_sa_sum - df_islanders.rolling_ga_sum) / df_islanders.rolling_sa_sum
df_islanders.to_csv('.//Team Frames//islanders.csv', index=False)

df_canucks = df[df.team == 'Vancouver Canucks']
df_canucks['rolling_points_5'] = df_canucks['points'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_points_10'] = df_canucks['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_canucks['rolling_points_20'] = df_canucks['points'].rolling(LONG_WINDOW).mean().shift()
df_canucks['rolling_goals_5'] = df_canucks['goals'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_goals_10'] = df_canucks['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_canucks['rolling_ga_5'] = df_canucks['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_ga_10'] = df_canucks['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_canucks['rolling_pp_5'] = df_canucks['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_pk_5'] = df_canucks['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_shots_5'] = df_canucks['shots'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_sa_5'] = df_canucks['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_fow_5'] = df_canucks['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_canucks['rolling_ga_sum'] = df_canucks['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_canucks['rolling_sa_sum'] = df_canucks['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_canucks['rolling_svp_5'] = (df_canucks.rolling_sa_sum - df_canucks.rolling_ga_sum) / df_canucks.rolling_sa_sum
df_canucks.to_csv('.//Team Frames//canucks.csv', index=False)

df_bruins = df[df.team == 'Boston Bruins']
df_bruins['rolling_points_5'] = df_bruins['points'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_points_10'] = df_bruins['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_bruins['rolling_points_20'] = df_bruins['points'].rolling(LONG_WINDOW).mean().shift()
df_bruins['rolling_goals_5'] = df_bruins['goals'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_goals_10'] = df_bruins['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_bruins['rolling_ga_5'] = df_bruins['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_ga_10'] = df_bruins['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_bruins['rolling_pp_5'] = df_bruins['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_pk_5'] = df_bruins['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_shots_5'] = df_bruins['shots'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_sa_5'] = df_bruins['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_fow_5'] = df_bruins['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_bruins['rolling_ga_sum'] = df_bruins['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_bruins['rolling_sa_sum'] = df_bruins['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_bruins['rolling_svp_5'] = (df_bruins.rolling_sa_sum - df_bruins.rolling_ga_sum) / df_bruins.rolling_sa_sum
df_bruins.to_csv('.//Team Frames//bruins.csv', index=False)

df_panthers = df[df.team == 'Florida Panthers']
df_panthers['rolling_points_5'] = df_panthers['points'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_points_10'] = df_panthers['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_panthers['rolling_points_20'] = df_panthers['points'].rolling(LONG_WINDOW).mean().shift()
df_panthers['rolling_goals_5'] = df_panthers['goals'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_goals_10'] = df_panthers['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_panthers['rolling_ga_5'] = df_panthers['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_ga_10'] = df_panthers['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_panthers['rolling_pp_5'] = df_panthers['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_pk_5'] = df_panthers['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_shots_5'] = df_panthers['shots'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_sa_5'] = df_panthers['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_fow_5'] = df_panthers['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_panthers['rolling_ga_sum'] = df_panthers['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_panthers['rolling_sa_sum'] = df_panthers['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_panthers['rolling_svp_5'] = (df_panthers.rolling_sa_sum - df_panthers.rolling_ga_sum) / df_panthers.rolling_sa_sum
df_panthers.to_csv('.//Team Frames//panthers.csv', index=False)

df_jets = df[df.team == 'Winnipeg Jets']
df_jets['rolling_points_5'] = df_jets['points'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_points_10'] = df_jets['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_jets['rolling_points_20'] = df_jets['points'].rolling(LONG_WINDOW).mean().shift()
df_jets['rolling_goals_5'] = df_jets['goals'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_goals_10'] = df_jets['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_jets['rolling_ga_5'] = df_jets['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_ga_10'] = df_jets['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_jets['rolling_pp_5'] = df_jets['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_pk_5'] = df_jets['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_shots_5'] = df_jets['shots'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_sa_5'] = df_jets['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_fow_5'] = df_jets['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_jets['rolling_ga_sum'] = df_jets['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_jets['rolling_sa_sum'] = df_jets['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_jets['rolling_svp_5'] = (df_jets.rolling_sa_sum - df_jets.rolling_ga_sum) / df_jets.rolling_sa_sum
df_jets.to_csv('.//Team Frames//jets.csv', index=False)

df_goldenknights = df[df.team == 'Vegas Golden Knights']
df_goldenknights['rolling_points_5'] = df_goldenknights['points'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_points_10'] = df_goldenknights['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_goldenknights['rolling_points_20'] = df_goldenknights['points'].rolling(LONG_WINDOW).mean().shift()
df_goldenknights['rolling_goals_5'] = df_goldenknights['goals'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_goals_10'] = df_goldenknights['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_goldenknights['rolling_ga_5'] = df_goldenknights['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_ga_10'] = df_goldenknights['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_goldenknights['rolling_pp_5'] = df_goldenknights['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_pk_5'] = df_goldenknights['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_shots_5'] = df_goldenknights['shots'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_sa_5'] = df_goldenknights['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_fow_5'] = df_goldenknights['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_goldenknights['rolling_ga_sum'] = df_goldenknights['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_goldenknights['rolling_sa_sum'] = df_goldenknights['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_goldenknights['rolling_svp_5'] = (df_goldenknights.rolling_sa_sum - df_goldenknights.rolling_ga_sum) / df_goldenknights.rolling_sa_sum
df_goldenknights.to_csv('.//Team Frames//goldenknights.csv', index=False)

df_kraken = df[df.team == 'Seattle Kraken']
df_kraken['rolling_points_5'] = df_kraken['points'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_points_10'] = df_kraken['points'].rolling(MEDIUM_WINDOW).mean().shift()
df_kraken['rolling_points_20'] = df_kraken['points'].rolling(LONG_WINDOW).mean().shift()
df_kraken['rolling_goals_5'] = df_kraken['goals'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_goals_10'] = df_kraken['goals'].rolling(MEDIUM_WINDOW).mean().shift()
df_kraken['rolling_ga_5'] = df_kraken['goals_against'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_ga_10'] = df_kraken['goals_against'].rolling(MEDIUM_WINDOW).mean().shift()
df_kraken['rolling_pp_5'] = df_kraken['power_play'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_pk_5'] = df_kraken['penalty_kill'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_shots_5'] = df_kraken['shots'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_sa_5'] = df_kraken['shots_against'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_fow_5'] = df_kraken['FOWp'].rolling(SHORT_WINDOW).mean().shift()
df_kraken['rolling_ga_sum'] = df_kraken['goals_against'].rolling(SHORT_WINDOW).sum().shift()
df_kraken['rolling_sa_sum'] = df_kraken['shots_against'].rolling(SHORT_WINDOW).sum().shift()
df_kraken['rolling_svp_5'] = (df_kraken.rolling_sa_sum - df_kraken.rolling_ga_sum) / df_kraken.rolling_sa_sum
df_kraken.to_csv('.//Team Frames//kraken.csv', index=False)

merged_df_test = merged_df.copy()
df_devils.head(10)


<ipython-input-12-25b8aea8f3d8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_devils['rolling_points_5'] = df_devils['points'].rolling(SHORT_WINDOW).mean().shift()
<ipython-input-12-25b8aea8f3d8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_devils['rolling_points_10'] = df_devils['points'].rolling(MEDIUM_WINDOW).mean().shift()
<ipython-input-12-25b8aea8f3d8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,team,date,points,goals,goals_against,power_play,penalty_kill,shots,shots_against,FOWp,...,rolling_ga_5,rolling_ga_10,rolling_pp_5,rolling_pk_5,rolling_shots_5,rolling_sa_5,rolling_fow_5,rolling_ga_sum,rolling_sa_sum,rolling_svp_5
0,New Jersey Devils,2005-10-05,2,5,1,25.0,90.9,37.0,37.0,57.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,New Jersey Devils,2005-10-07,0,2,5,22.2,100.0,28.0,33.0,48.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,New Jersey Devils,2005-10-08,2,3,2,20.0,87.5,27.0,35.0,43.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,New Jersey Devils,2005-10-13,0,1,4,0.0,87.5,21.0,23.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,New Jersey Devils,2005-10-15,0,1,6,0.0,50.0,28.0,25.0,43.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,New Jersey Devils,2005-10-18,2,4,3,42.9,100.0,34.0,24.0,60.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,New Jersey Devils,2005-10-20,2,6,3,33.3,80.0,36.0,33.0,50.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,New Jersey Devils,2005-10-22,0,3,4,0.0,85.7,35.0,28.0,49.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,New Jersey Devils,2005-10-26,0,3,6,0.0,75.0,23.0,39.0,55.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,New Jersey Devils,2005-10-28,2,3,2,33.3,80.0,27.0,27.0,47.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
